**Instalando Bibliotecas**

Para a raspagem de dadoa foi necessário instalar duas bibliotecas que não eram nativas, a Requests e a bs4

In [1]:
! pip install requests
! pip install bs4

In [2]:
import requests
import csv
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

In [3]:
with open('alagamentos.csv', 'a', newline='', encoding='utf-8') as csvfile:
    campos_head = ['Data', 'Sub-prefeitura', 'Pontos de Alagamento', 'Hora de Inicio', 'Hora de Termino', 'Local']
    writer = csv.DictWriter(csvfile, fieldnames=campos_head, delimiter=';')
    writer.writeheader()

In [6]:
data_x = datetime(year= 2012, month= 1, day= 1).date()
data_y = datetime(year= 2012, month= 12, day= 31).date()

while data_x <= data_y:
    ano = data_x.year
    mes = data_x.month
    dia = data_x.day
    data_completa = data_x.strftime("%d-%m-%Y")

    url = f'https://www.cgesp.org/v3/alagamentos.jsp?dataBusca={dia}%2F{mes}%2F{ano}&enviaBusca=Buscar'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36'}
    site = requests.get(url, headers = headers)

    soup = BeautifulSoup(site.text, 'html.parser')

    sub_prefeituras = soup.find_all('td' , class_='bairro arial-bairros-alag linha-pontilhada')
    pontos_alagamento = soup.find_all('td' , class_='total-pts arial-bairros-alag')
    infos_alagamento = soup.find_all('li' , class_='arial-descr-alag col-local')
    contador = 0

    for i in range(len(sub_prefeituras)):
      sub_pref = (sub_prefeituras[i].get_text().strip())
      pts_alag = (int(pontos_alagamento[i].get_text().replace(" pts.","").strip()))
      for j in range(pts_alag):
          hora_inicio = (infos_alagamento[contador].get_text().strip()[3:8])
          hora_fim = (infos_alagamento[contador].get_text().strip()[11:16])
          local = (infos_alagamento[contador].get_text().strip()[16:])
          contador = contador + 1

          with open('alagamentos.csv', 'a', newline='', encoding='utf-8') as csvfile:
              writer = csv.DictWriter(csvfile, fieldnames=campos_head, delimiter=';')
              writer.writerow({'Data' : data_completa,
                               'Sub-prefeitura': sub_pref,
                               'Pontos de Alagamento' : pts_alag,
                               'Hora de Inicio' : hora_inicio,
                               'Hora de Termino' : hora_fim,
                               'Local' : local})

    data_x = data_x + timedelta(days = 1)